# Bicycle Accidents x Geo Data x Strava Exposure Merge


#### Import some standard libraries and helper scripts:

In [ ]:
import sys
from pathlib import Path

# Ensure project root is importable when running from notebooks/
_project_root = Path.cwd().resolve()
if not (_project_root / "src").exists() and (_project_root.parent / "src").exists():
    _project_root = _project_root.parent
sys.path.insert(0, str(_project_root))

import datetime as dt

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

# Optional: reload project modules without restarting the kernel
import importlib
import src.accidents as accidents
import src.segments as segments
import src.strava_exposure as strava_exposure
import src.panels as panels
import src.nodes as nodes
for _m in (accidents, segments, strava_exposure, panels, nodes):
    importlib.reload(_m)

from src.accidents import (
    ACCIDENT_COLUMNS_EN,
    assign_accidents_to_nearest_segment,
    load_accidents_raw,
    prepare_accidents_bike_berlin,
)
from src.segments import load_segment_geometry
from src.strava_exposure import (
    build_exposure_panel_segment_year_month,
    column_stability_summary,
    load_strava_berlin_data,
)
from src.panels import (
    aggregate_accidents_segment_year_month_rich,
    build_core_risk_panel,
    merge_exposure_and_accidents,
    sanity_check_merge,
)
from src.nodes import (
    assign_accidents_to_nearest_crossing,
    build_node_exposure_panel_from_segments,
    build_node_risk_panel,
    build_nodes_from_segment_endpoints,
    cluster_nodes_snap_grid,
    select_crossings_by_degree,
)


In [ ]:
import sys
print(sys.executable)

In [ ]:
# Kernel cleanup: drop old variables from pre-refactor runs
# (Equivalent to a kernel restart for the pipeline variables.)
import gc
import sys

# Ensure the removed module isn't lingering in memory
sys.modules.pop("src.merge_datasets", None)

# Drop previously computed objects/dataframes so we don't keep stale types around
for _name in [
    "seg",
    "clustering",
    "accidents_raw",
    "accidents_bike_berlin",
    "segment_geo_gdf",
    "segment_static",
    "strava_berlin_data",
    "summary_df",
    "final_exposure_ym",
    "accidents_agg_ym_rich",
    "merged_accidents_strava_ym",
    "core_panel",
    "nodes_raw",
    "node_points",
    "crossings_gdf",
    "crossing_ids",
    "segment_node_map",
    "node_exposure_ym",
    "node_panel_ym",
    "acc_node",
    "acc_node_ym",
    "joined_nearest_unique",
    "stats",
    "merge_keys",
    "min_year",
    "max_year",
    "out_dir",
    "out_path",
]:
    globals().pop(_name, None)

gc.collect()
print("kernel_cleanup_done")


## Bicycle data for Berlin

In [ ]:
import gc

accidents_raw = load_accidents_raw()
print("Raw accidents shape:", accidents_raw.shape)

accident_columns_en = ACCIDENT_COLUMNS_EN

accidents_bike_berlin = prepare_accidents_bike_berlin(accidents_raw, column_map=accident_columns_en)
print(f"Filtered to bicycle accidents in Berlin -> shape: {accidents_bike_berlin.shape}")

# Free big raw dataframe early to keep memory low for Strava aggregation
del accidents_raw
gc.collect()

accidents_bike_berlin.head()


## Rename columns to English

In [ ]:
# Column map is now maintained in src.accidents
accident_columns_en = ACCIDENT_COLUMNS_EN

# (Optional) quick per-column uniqueness scan
for col in accidents_bike_berlin.columns:
    uniq_cnt = accidents_bike_berlin[col].nunique(dropna=True)
    first_vals = accidents_bike_berlin[col].head(5).tolist()
    print(f"{col}: uniques={uniq_cnt}; first5={first_vals}")


In [ ]:
seg = load_segment_geometry(canonical_crs="EPSG:32633")
CANONICAL_CRS = seg.canonical_crs

segment_geo_gdf = seg.segments_gdf
segment_static = seg.segment_static

segment_geo_gdf.head()


## Spatial Join: Accidents with Strava data (code from Luise and Eric) + edited by Tobi to achieve canonical geometry data


### Attempt 2: Use sjoin_nearest to assign exactly one (the nearest) segment to each accident
Challenges:
* need to find the right maximum distance so accidents that are not on a segment are not assigned to one.
* assigns two segments if their distance is equal

In [ ]:
# Assign each accident to exactly one nearest segment (within max_distance)
joined_nearest_unique = assign_accidents_to_nearest_segment(
    accidents_bike_berlin,
    segment_geo_gdf,
    canonical_crs=CANONICAL_CRS,
    max_distance_m=10,
)

print(f"Total accidents: {len(accidents_bike_berlin)}")
print(f"Total bike network Strava segments: {len(segment_geo_gdf)}")
print(f"Unique Strava segments in matched dataset: {joined_nearest_unique['counter_name'].nunique()}")
print(f"Accidents assigned to segments: {len(joined_nearest_unique)}")
print(f"Ratio of assigned accidents: {len(joined_nearest_unique) / len(accidents_bike_berlin):.2%}")

joined_nearest_unique.head()


## Strava data (bicycle network traffic, other features - daily)

1. We need to aggregate this df to the same granularity as in Accidents data (segment, year, month, weekday) to join. 
2. We can not join only by geo data, as Accidents don't have date column, but Strava contains daily info (eg specific traffic volume or weather on specific day)
3. We can not just calculate mean of all columns in Strava data by year-month-etc... as we also have categorial features (for example `infrastructure_bicyclelane_type`) and some features are constant over time for segment (eg `infrastructure_max_speed` in dataset is constant for segment for all dates)

In [ ]:
strava_berlin_data = load_strava_berlin_data()
strava_berlin_data.columns.tolist()


### Which data types we have as features?

Results:

1. Mostly we have numerical features, but also categorical ones like `'infrastructure_bicyclelane_type'` - we will check if we need to aggregate them somehow or they are contstant over time.
2. Analysis shows:
    - **Numeric columns (111)**: Traffic counts, speeds, socioeconomic indicators, weather data
    - **Categorical columns**: Infrastructure types, activity types, street properties
    - **Boolean columns (8)**: Holiday flags, weekend indicators, data quality flags
3. **Key finding**: All connectivity and infrastructure columns are constant per segment, so they only need to be taken once per segment. Socioeconomic, motorized, strava, and weather columns vary over time and require aggregation by year-month-weekday.

In [ ]:
df = strava_berlin_data
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=["number"]).columns.tolist()
bool_cols = df.select_dtypes(include=["bool"]).columns.tolist()

print("Numeric:", len(numeric_cols))
print(numeric_cols)
print("\nCategorical:", len(categorical_cols))
print(categorical_cols)
print("\nBool:", len(bool_cols))
print(bool_cols)


### Check which features we have contstant for one segment over time, so we don't need to aggregate them futher

In [ ]:
summary_df = column_stability_summary(strava_berlin_data, group_col="counter_name")
summary_df.head(10)


In [ ]:
# tag dtypes
col_dtype = strava_berlin_data.dtypes
summary_df["dtype_bucket"] = summary_df["column"].map(
    lambda c: "bool" if col_dtype[c].name == "bool"
    else "numeric" if np.issubdtype(col_dtype[c], np.number)
    else "categorical"
)

# overall constant/varying summary
overall_stats = {
    "total_columns": len(summary_df),
    "constant_columns": int((summary_df["segments_varying"] == 0).sum()),
    "varying_columns": int((summary_df["segments_varying"] > 0).sum()),
}
overall_stats["percent_constant"] = round(
    overall_stats["constant_columns"]
    / max(overall_stats["total_columns"], 1)
    * 100,
    1,
 )

print("Overall column stability:")
for key, value in overall_stats.items():
    print(f"  {key}: {value}")

# dtype-level statistics
dtype_counts = (
    summary_df
    .groupby(["dtype_bucket"])
    .agg(
        total_cols=("column", "count"),
        constant_cols=("segments_varying", lambda s: (s == 0).sum()),
        varying_cols=("segments_varying", lambda s: (s > 0).sum()),
    )
)

dtype_counts["percent_constant"] = (
    dtype_counts["constant_cols"] / dtype_counts["total_cols"] * 100
).round(1)

display(dtype_counts.sort_values("percent_constant", ascending=False))

### Result: All connectivity and infrastructure columns are constant per segment. Socioeconomic, Motorized and weather columns vary, so we need to aggregate them.

- Connectivity (7/7 constant, 2 bool, 5 numeric): treat as static attributes per segment; just carry a single value (e.g., first).
- Infrastructure (58/58 constant, 1 bool, 10 categorical, 47 numeric): fully static; keep one value per segment, no temporal aggregation needed.
**- Other (14 cols, 5 constant/9 varying; 5 bool/6 cat/3 num): mixed bag—decide column by column; reassign misfiled cols if any.**
- Motorized (12/12 varying, all numeric): fully time-varying; aggregate over your time buckets (sum for counts, mean for speeds).
- Socioeconomic (17/17 varying, numeric): varies across time in the data; aggregate over your time buckets (sum for counts, mean for speeds).
**- Strava (19/19 varying; 1 categorical, 18 numeric): counts/speeds should be summed/averaged per time bucket; handle the single categorical (strava_activity_type) via ????**
- Weather (9/9 varying, numeric): time-varying; aggregate with mean (or min/max if useful).



## Aggregation of Berlin Strava data 
1. Aggregation keys: counter_name (segment), year, month, weekday (to align with accidents).
2. Constant features stay as-is (no aggregation) since they don’t vary over time.

### This code execution can take a while, on Liaisan's pc ~13 minutes.

# Segment level risk

In [ ]:
# --- 1. Build exposure panel from Strava/sensor data (segment x year x month) ---
final_exposure_ym = build_exposure_panel_segment_year_month(
    strava_berlin_data,
    segment_static=segment_static,
    summary_df=summary_df,
)

print("Exposure panel (segment–year–month) shape:", final_exposure_ym.shape)
final_exposure_ym.head()


## Aggregate accidents

In [ ]:
# --- 2. Build rich accident panel from Unfallatlas+segments (segment x year x month) ---
min_year = int(final_exposure_ym["year"].min())
max_year = int(final_exposure_ym["year"].max())

accidents_agg_ym_rich = aggregate_accidents_segment_year_month_rich(
    joined_nearest_unique,
    column_map=accident_columns_en,
    exposure_year_min=min_year,
    exposure_year_max=max_year,
)

print(
    "Rich accident aggregate (segment–year–month) shape:",
    accidents_agg_ym_rich.shape,
)
accidents_agg_ym_rich.head()


## Merge datasets

In [ ]:
# --- 3. Merge exposure and accident panels into a risk panel ---

merge_keys = ["counter_name", "year", "month"]

merged_accidents_strava_ym = merge_exposure_and_accidents(
    final_exposure_ym,
    accidents_agg_ym_rich,
    merge_keys=merge_keys,
 )

print(
    "Merged risk panel (segment–year–month) shape:",
    merged_accidents_strava_ym.shape,
 )

merged_accidents_strava_ym.head()

# Save geodataframe to parquet file
out_dir = _project_root / "data" / "merged"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "berlin_bike_accident_strava_panel.parquet"

gpd.GeoDataFrame(
    merged_accidents_strava_ym,
    geometry="geometry",
    crs=segment_geo_gdf.crs,
 ).to_parquet(
    out_path,
    index=False,
 )

## Sanity check of the merge

In [ ]:
stats = sanity_check_merge(
    merged_accidents_strava_ym=merged_accidents_strava_ym,
    accidents_agg_ym_rich=accidents_agg_ym_rich,
    final_exposure_ym=final_exposure_ym,
)

for k, v in stats.items():
    print(f"{k}: {v}")


# Playground, to create and test smaller dataset version. The smaller ones are created based on the completly merged version.

In [ ]:
core_panel = build_core_risk_panel(merged_accidents_strava_ym)
print("Core panel shape:", core_panel.shape)
core_panel.head(20)

# Save geodataframe to parquet file
out_dir = _project_root / "data" / "merged"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "berlin_bike_accident_strava_risk_core_panel.parquet"

gpd.GeoDataFrame(
    core_panel,
    geometry="geometry",
    crs=segment_geo_gdf.crs,
 ).to_parquet(
    out_path,
    index=False,
 )

In [ ]:
# find unique numbers of accidents in core_panel
unique_accident_counts = core_panel["total_accidents"].nunique()
print(f"Unique accident counts in core panel: {unique_accident_counts}")

# display those unique counts
print("Unique accident counts:", core_panel["total_accidents"].unique())




In [ ]:
# sum up the accidents per segment to verfiy total accidents per segment
accidents_per_segment = (
    core_panel
    .groupby("counter_name", as_index=False)["total_accidents"]
    .sum()
    .rename(columns={"total_accidents": "total_accidents_segment"})
)

# find segments with highest total accidents
top_segments = accidents_per_segment.sort_values("total_accidents_segment", ascending=False).head(10)
print("Top 10 segments by total accidents:")
display(top_segments)

# Crossing (junction) risk

### Build nodes (junction candidates) from segment endpoints

In [ ]:
nodes_raw = build_nodes_from_segment_endpoints(
    segment_geo_gdf,
    counter_col="counter_name",
)

nodes_raw.head()

### Cluster endpoints into nodes (snap grid)

In [ ]:
clustering = cluster_nodes_snap_grid(
    nodes_raw,
    tol_m=2,
    counter_col="counter_name",
)

nodes_raw = clustering.nodes_raw
node_points = clustering.node_points
segment_node_map = clustering.segment_node_map

print("Nodes (raw endpoints):", len(nodes_raw))
print("Nodes (clustered):", len(node_points))
segment_node_map.head()

### Define crossings (nodes with degree $\geq$ 3)

In [ ]:
crossing_ids = select_crossings_by_degree(
    nodes_raw,
    min_degree=3,
    counter_col="counter_name",
)

crossings_gdf = node_points[node_points["node_id"].isin(crossing_ids)].copy()

print("Crossings (degree >= 3):", len(crossings_gdf))
crossings_gdf.head()

### Assign accidents to nearest crossing

In [ ]:
acc_node, acc_node_ym = assign_accidents_to_nearest_crossing(
    joined_nearest_unique,
    crossings_gdf,
    max_distance_m=20,
)

print("Accidents assigned to crossings:", len(acc_node))
print("Accident groups (node×year×month):", len(acc_node_ym))
acc_node_ym.head()

### Build node-level exposure from segment flows

In [ ]:
node_exposure_ym = build_node_exposure_panel_from_segments(
    final_exposure_ym,
    segment_node_map,
    crossing_ids,
    trip_col="sum_strava_total_trip_count",
)

print("Node exposure (node×year×month) shape:", node_exposure_ym.shape)
node_exposure_ym.head()

### Combine into node-level risk panel

In [ ]:
node_panel_ym = build_node_risk_panel(
    node_exposure_ym,
    acc_node_ym,
    crossings_gdf,
)

print("Node panel (crossing x year x month) shape:", node_panel_ym.shape)
node_panel_ym.head()

# Save geodataframe to parquet file
out_dir = _project_root / "data" / "merged"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "berlin_bike_accident_node_panel.parquet"

node_panel_ym.to_parquet(
    out_path,
    index=False,
)

In [ ]:
# Look at the overall description of the dataset
node_panel_ym.describe(include="all")